In [1]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

#  ----------------------------------------------------------------------------------------------------------------------------

# PART - 1 : CAPTURING FACE DATA OF USER

#  ----------------------------------------------------------------------------------------------------------------------------

In [2]:
# cascade classifiers are basically used for face features detection

In [3]:
face_classifier=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

#  ----------------------------------------------------------------------------------------------------------------------------

In [4]:
# Lets extract the features of face

In [5]:
def face_ext(img):
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)     # As the image captured will be coloured , we need to convert it into grayscale
    faces=face_classifier.detectMultiScale(gray,1.3,5)      # (1.3)== scaling factor, (5) == minimum neighbours  # lies from 3 to 6,low means low accuracy
    
    if faces is():     # if face has no value it will return none
        return None
    
    for(x,y,w,h) in faces:       # if face has some value then capture the cordinates   x,y,height,width
        cropped_faces=img[y:y+h,x:x+w]    # we gonna capture the face from original image(img) by cordinates given by cascade classfier(faces)
    
    return cropped_faces


#  ----------------------------------------------------------------------------------------------------------------------------

In [6]:
# Configuring the camera

In [7]:
capture=cv2.VideoCapture(0)

#  ----------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Setting up a loop to capture user face

count=0
while True:
    ret,frame=capture.read()     # capturing the image from camera
    
    if face_ext(frame) is not None:     #  if the frame is not empty
        count=count+1     # keeping a count 
        
        # now we want the size of camera equal to size of our face
        face=cv2.resize(face_ext(frame),(200,200))
        face=cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)    # conveting the image to grayscale
        
        # Saving the captured faces to a folder
        file_name_path="G:/Datasets/OpenCv/my_face_data/Tarun"+str(count)+".JPG"
        
        cv2.imwrite(file_name_path,face)
        
        # to keep track of number of images
        cv2.putText(face,str(count),(50,50),cv2.FONT_HERSHEY_DUPLEX,1,(0,255,0),2)
        
        # To show the faces
        cv2.imshow("Face Cropper",face)
    
    else:
        print("FACE NOT FOUND : PLEASE ADJUST FACE OR CAMERA")
        pass
    
    if cv2.waitKey(1)==13 or count==200:    # To break the loop when Enter key is pressed or 150 images collected
        break
capture.release()
cv2.destroyAllWindows()
print("Face Sample Collection Complete")

#  ----------------------------------------------------------------------------------------------------------------------------

# PART - 2 : TRAINING THE MODEL ON CAPTURED FACE DATA

#  ----------------------------------------------------------------------------------------------------------------------------

In [4]:
import os
import path
from os import listdir                   # list dir is used to fetch data from directory
from os.path import isfile, join

#  ----------------------------------------------------------------------------------------------------------------------------

In [5]:
# Reading the data path
data_dir="G:/Datasets/OpenCv/my_face_data/"     # remember to add "/" at the end or images will be empty


myfiles=[f for f in listdir(data_dir) if isfile(join(data_dir,f))]     # List comprehension


#  ----------------------------------------------------------------------------------------------------------------------------

In [6]:
# Creating Training images and labels

In [10]:
train_images,train_labels=[],[]

In [11]:
for i, files in enumerate(myfiles):
    image_path = data_dir + myfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    train_images.append(np.asarray(images, dtype=np.uint8))
    train_labels.append(i)

#  ----------------------------------------------------------------------------------------------------------------------------

In [12]:
# Building the model

In [13]:
model = cv2.face.LBPHFaceRecognizer_create()

In [14]:
model.train(np.asarray(train_images), np.asarray(train_labels))
print("Model Trained Successfully")

Model Trained Successfully


#  ----------------------------------------------------------------------------------------------------------------------------

# PART - 3 : PREDICTIONS

#  ----------------------------------------------------------------------------------------------------------------------------

In [6]:
# defining a function to detect face

def face_det(img,size=0.5):
    # converting image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    
    # detecting faces
    faces=face_classifier.detectMultiScale(gray,1.3,5)
    
    if faces is():    # if there are no faces present
        return img,[]    # return empty image
    
    # setting up a loop for image ROI (region of interest) 
    for(x,y,w,h) in faces:
        # drawing a contour on face
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        
        # Picking up ROI from contour
        roi=img[y:y+h,x:x+w]
        
        # resize the area
        roi=cv2.resize(roi,(200,200))
        
        # ending up the function to return image and roi
        return img,roi
    

#  ----------------------------------------------------------------------------------------------------------------------------

In [25]:
# initializing the camera
cap = cv2.VideoCapture(0)


while True:

    ret, frame = cap.read()
    
    #feeding the captured image to our previously built function
    image, face = face_det(frame)
    
    # For error handling
    try:        
        
        # converting the image into grayscale
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        
        # prediction from model
        result = model.predict(face)
        
        # to calculate confidence levels
        if result[1] < 500:
            confidence = int(100*(1-(result[1])/300))
            display_string = str(confidence)+'% Confidence it is user'
        
        # To put it in text
        cv2.putText(image,display_string,(100,120), cv2.FONT_HERSHEY_COMPLEX,1,(250,120,255),2)

        # Checking Confidence 

        if confidence > 75:
            cv2.putText(image, "ACCESS GRANTED", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow('Face Cropper', image)

        else:
            cv2.putText(image, "ACCESS DENIED", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
            cv2.imshow('Face Cropper', image)


    # Now we will handle if there is no face in the screen
    except:
        cv2.putText(image, "ADJUST CAMERA", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2)
        cv2.imshow('Face Cropper', image)
        pass
    
    
    # Condition to close program
    if cv2.waitKey(1)==13:
        break


cap.release()
cv2.destroyAllWindows()

#  ----------------------------------------------------------------------------------------------------------------------------

#  ----------------------------------------------------------------------------------------------------------------------------